In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F

import time
import re, collections

import matplotlib.pyplot as plt

import tqdm
import os

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device utilisé : {device}")

Device utilisé : cuda


---
---
# Token 

In [19]:
class Tokenizer():
    """ This class is used to tokenize the input text and create a vocabulary of tokens. """
    def __init__(self, text, num_merges):
        self.text = text
        self.num_merges = num_merges

    def initialize_vocabulary(self, text):
        """ Initialize the vocabulary from the input text. """
        vocab = collections.defaultdict(int)
        words = text.strip().split()
        for word in words:
            vocab[' '.join(list(word)) + ' '] += 1
        return vocab
    
    def get_tokens_and_frequencies(self, vocab):
        """ Get the tokens and how often they occur in the vocabulary. """
        tokens = collections.defaultdict(int)
        for word, freq in vocab.items():
            word_tokens = word.split()
            for token in word_tokens:
                tokens[token] += freq
        return tokens
    
    def get_pairs_and_counts(self, vocab):
        """ Get the pairs of tokens and how often they occur in the vocabulary. """
        pairs = collections.defaultdict(int)
        for word, freq in vocab.items():
            symbols = word.split()
            for i in range(len(symbols)-1):
                pairs[symbols[i],symbols[i+1]] += freq
        return pairs
    
    def merge_pair_in_vocabulary(self, pair, vocab_in):
        """ Merge the most frequent pair of tokens in the vocabulary. """
        vocab_out = {}
        bigram = re.escape(' '.join(pair))
        p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
        for word in vocab_in:
            word_out = p.sub(''.join(pair), word)
            vocab_out[word_out] = vocab_in[word]
        return vocab_out
    
    def tokenize(self):
        """ Tokenize the input text. """
        if os.path.exists('./data/tokens_{}merges.pt'.format(self.num_merges)):
            print("Loading tokenized data from file")
            tokens = torch.load('./data/tokens_{}merges.pt'.format(self.num_merges),weights_only=False)
            vocab = torch.load('./data/vocab_{}merges.pt'.format(self.num_merges),weights_only=False)
            return tokens, vocab
        
        # Initialize the vocabulary from the input text
        vocab = self.initialize_vocabulary(self.text)

        # Merge the most frequent pair of tokens num_merges times
        with tqdm.tqdm(range(self.num_merges), position=0, leave=True) as pbar:
            for i in pbar:
                tokens = self.get_tokens_and_frequencies(vocab)
                pairs = self.get_pairs_and_counts(vocab)
                if not pairs:
                    break
                most_frequent_pair = max(pairs, key=pairs.get)
                vocab = self.merge_pair_in_vocabulary(most_frequent_pair, vocab)
                pbar.set_description(f'Num merges: {i + 1}')

        # Find the tokens and how often they occur in the vocabulary one last time
        tokens = self.get_tokens_and_frequencies(vocab)

        tokens[' '] = self.text.count(' ')
        tokens['\n'] = self.text.count('\n')

        torch.save(tokens, './data/tokens_{}merges.pt'.format(self.num_merges))
        torch.save(vocab, './data/vocab_{}merges.pt'.format(self.num_merges))

        return tokens, vocab


    def tokenize_text(self, tokens):
        """ Tokenize the input text using the tokens. """
        # Load tokenized data if it exists
        if os.path.exists('./data/tokenized_{}merges.pt'.format(self.num_merges)):
            print("Loading tokenized data from file")
            tokenized_text = torch.load('./data/tokenized_{}merges.pt'.format(self.num_merges),weights_only=False)
            return tokenized_text

        token_list = list(tokens.keys())
        tokenized_text = []  # List to store the tokenized text
        i = 0  # Index to keep track of the current position in the text

        print("Nombre de tokens trouvés :", len(token_list))

        with tqdm.tqdm(total=len(self.text), position=0, leave=True) as pbar:
            pbar.set_description("Tokenizing text")
            while i < len(self.text):
                match = None
                # loop through the tokens to find the longest match
                for token in sorted(token_list, key=len, reverse=True):
                    if self.text[i:i+len(token)] == token: 
                        match = token
                        break
                    
                if match:  
                    tokenized_text.append(match)
                    i += len(match)  # Move the index to the end of the token
                    pbar.update(len(match))
                else:
                    print("Token non trouvé pour le texte restant :", self.text[i:])
                    break 

        # Save the tokenized text
        torch.save(tokenized_text, './data/tokenized_{}merges.pt'.format(self.num_merges))
        return tokenized_text 

In [20]:
data = open('./shakespeare.txt', 'r').read()

In [21]:
tokenizer = Tokenizer(data, 1000)
tokens, vocab = tokenizer.tokenize()
tokenized_text = tokenizer.tokenize_text(tokens)

Num merges: 1000: 100%|██████████| 1000/1000 [01:29<00:00, 11.19it/s]


Nombre de tokens trouvés : 1014


Tokenizing text: 100%|██████████| 1115394/1115394 [00:58<00:00, 19223.29it/s]


In [22]:
print(len(tokens))

1014
